In [1]:
%matplotlib notebook
%run functions.py
%run dataset.py

In [302]:
#for each tracing, calculate qom(lh),qom(rh),select whichever is greater, and make dataframe from it
# p = numpy.zeroes(4)
p = []
for i in tracings:
    x = qomlhrh(i)
    m = []
    if x['qomlh'] > x['qomrh']:
        n = []
        df = pd.DataFrame(getlhrh(i)['lh'])
        df = df - numpy.mean(df)
        a = numpy.array(df['LHX'])
        b = numpy.array(df['LHZ'])
        c = numpy.diff(getlhrh(i)['lh']['LHX']) + numpy.diff(getlhrh(i)['lh']['LHZ']) + numpy.diff(getlhrh(i)['lh']['LHY'])
        d = numpy.diff(c)
        n = [a,b,c,d]
    else:
        n = []
        df = pd.DataFrame(getlhrh(i)['rh'])
        df = df - numpy.mean(df)
        a = numpy.array(df['RHX'])
        b = numpy.array(df['RHZ'])
        c = numpy.diff(getlhrh(i)['rh']['RHX']) + numpy.diff(getlhrh(i)['rh']['RHZ']) + numpy.diff(getlhrh(i)['rh']['RHY'])
        d = numpy.diff(c)
        n = [a,b,c,d]
    g = pd.DataFrame(n)
    g = numpy.array(g.T)
    p.append(g)

In [313]:
p.shape

(794, 600, 4)

In [314]:
numpy.save('pickle',p, allow_pickle = True)

In [316]:
melodies = []
for i in tracings:
    x = int(returnDetails(i)['melID'])
    if x>16:
        x = x-16
    x = x-1
    melodies.append(x)

In [318]:
melodies= pd.DataFrame(melodies)

In [319]:
melodies.to_csv('melodies794')